# <center>Определение параметров адаптивных моделей</center> 

Параметры адаптивных моделей ($\alpha, \beta, \gamma, \phi$) выбираются таким образом, чтобы ошибка при аппроксимации полученной моделью была как можно меньше, т.е. необходимо решить задачу оптимизации.

## Оптимизация в <a href='https://www.scipy.org/'>SciPy</a>

Решать задачи оптимизации позволяет библиотека SciPy. Алгоритмы оптимизации лежат в модуле `scipy.optimize`. Найдем минимум следующей функции:

$$f(x)=x^2+2x^4-3x.$$

In [3]:
def fun(x):
    return x**2 + 2*x**4 - 3*x

Воспользуемся функцией `minimize` из указанного модуля. В данной функции реализованы численные методы решения задач оптимизации. В качестве первого аргумента `minimize` принимает функцию, минимум которой необходимо найти, а в качестве второго аргумента &ndash; начальное значение переменных.

In [4]:
from scipy.optimize import minimize

In [5]:
result = minimize(fun, 0)
result

      fun: -1.1810421376911344
 hess_inv: array([[0.09247045]])
      jac: array([-3.03983688e-06])
  message: 'Optimization terminated successfully.'
     nfev: 21
      nit: 4
     njev: 7
   status: 0
  success: True
        x: array([0.60670554])

Значение целевой функции и оптимальное значение $x$:

In [6]:
result.fun, result.x

(-1.1810421376911344, array([0.60670554]))

Функция нескольких переменных может быть передана в `minimize` двумя способами:

In [7]:
def fun1(x1, x2):
    return x1**2 + 2*x1*x2 + x2**2

In [8]:
minimize(lambda x: fun1(x[0], x[1]), (2,2))

      fun: 2.8552400645473715e-15
 hess_inv: array([[ 0.62499999, -0.375     ],
       [-0.375     ,  0.62500001]])
      jac: array([-9.19677317e-08, -9.19677317e-08])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 2
     njev: 3
   status: 0
  success: True
        x: array([ 5.67418834e-08, -1.10176330e-07])

In [9]:
fun2 = lambda x: x[0]**2 + 2*x[0]*x[1] + x[1]**2

In [10]:
minimize(fun2, (2,2))

      fun: 2.8552400645473715e-15
 hess_inv: array([[ 0.62499999, -0.375     ],
       [-0.375     ,  0.62500001]])
      jac: array([-9.19677317e-08, -9.19677317e-08])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 2
     njev: 3
   status: 0
  success: True
        x: array([ 5.67418834e-08, -1.10176330e-07])

Можно указать ограничения на значения переменных с помощью опций `constraints` и `bounds`.

In [11]:
fun3 = lambda x: x[0] + x[1]

In [12]:
# x1 >= 0, x2 >= 0
bnds = ((0,None),(0,None))

# 3*x1 + 2*x2 >= 7
# x1 + 2*x2 >= 6
cons = ({'type': 'ineq', 'fun': lambda x: 3*x[0] + 2*x[1] - 7},
        {'type': 'ineq', 'fun': lambda x: x[0] + 2*x[1] - 6})

In [13]:
minimize(fun3, (2,2), bounds=bnds, constraints=cons)

     fun: 3.2500000000000004
     jac: array([1., 1.])
 message: 'Optimization terminated successfully.'
    nfev: 12
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([0.5 , 2.75])

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from dateutil.relativedelta import relativedelta
passengers = pd.read_csv('ausair.csv', index_col='Year', parse_dates=True)

<div class="alert alert-info">

<h3> Задание 1</h3>
<p></p>
<ul>1. Определите функцию, которая принимает на вход временной ряд и параметры модели Хольта для линейного тренда (см. задание 2 с предыдущего занятия). Данная функция должна возвращать среднеквадратичную ошибку модели.</ul>
<ul>2. Определите оптимальные параметры $\alpha$ и $\beta$ для ряда объемов пассажирских авиаперевозок в Австралии (файл ausair.csv). Постройте модель Хольта с найденными параметрами и получите прогноз на 10 лет вперед.</ul>
<p></p>
  
</div>

In [2]:
alpha = 0.59
beta = 0.43

In [3]:
def holt(passengers):
    T = len(passengers)
    data = passengers.values.reshape(1,T)[0]
    y = np.array([data[0]])
    l = np.array([data[0]])
    b = np.array([data[1]-data[0]])
    for i in range(1, T):
        y = np.append(y, l[-1] + b[-1])
        l = np.append(l, alpha*data[i] + (1-alpha)*(l[-1]+b[-1]))
        b = np.append(b, beta*(l[-1]-l[-2]) + (1-beta)*b[-1])
    print(list(y))
    summ = 0
    summ += (sum((data-y)**2)/(T-1))**0.5
    return summ

In [4]:
holt(passengers)

[7.32, 7.33, 7.34, 7.738102, 9.2500733426, 10.982879986048381, 11.948896057401743, 11.950709716893797, 12.054405262109583, 12.591461820650855, 13.451375145753655, 14.2260307345681, 14.034709528622097, 13.724265026772809, 12.96365674372234, 13.09951563178929, 14.177987660111794, 15.598655722353662, 17.320406671111705, 19.551371387641463, 17.150391000374025, 17.706969844799485, 21.531690921388293, 24.895829576033496, 28.58479546099833, 29.643701865378635, 31.016017327927997, 32.07869907147791, 32.74134763189942, 31.917356647459364, 32.3140329623785, 33.13414608893992, 34.089135608066044, 39.0002416071414, 43.01837677103052, 44.42987000141462, 46.872366206513206, 49.24458534401123, 51.136844888609765, 53.63066675365488, 52.878244562921125, 60.79880881910721, 66.30081436673599, 70.34165203642287, 72.54065835935403, 72.27352992598767, 72.51079872608439]


2.182386397982945

AttributeError: 'DataFrame' object has no attribute 'reshape'

<div class="alert alert-info">

<h3> Задание 2</h3>
<p></p>
<ul>1. Для модели Хольта с аддитивным затухающим трендом (задание 3 с предыдущего занятия) проделайте те же шаги, что и в задании 1. В данном случае кроме параметров $\alpha$ и $\beta$ необходимо также определить оптимальный параметр $\phi$.</ul>
<ul>2. Сравните на графике аппроксимации и прогнозы с помощью модели для линейного тренда и модели с аддитивным затухающим трендом.</ul>
<p></p>
  
</div>